##Demo
Demo with telegram bot that converts your voice into Beyonce's by default but you can the voice that it converts to.

In [ ]:
!git clone https://github.com/EdvardOlsen/VoiceConversion.git
!chmod 777 VoiceConversion/setup.sh
!./VoiceConversion/setup.sh

In [ ]:
import nemo
# NeMo's ASR collection - this collections contains complete ASR models and
# building blocks (modules) for ASR
import nemo.collections.asr as nemo_asr
quartznet = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="QuartzNet15x5Base-En")
def recognise_quartznet(audio_file):
  return quartznet.transcribe(paths2audio_files=[audio_file])[0]

recognise_quartznet('../audio.wav')

In [1]:
cd Real-Time-Voice-Cloning

/content/Real-Time-Voice-Cloning


In [ ]:
from IPython.display import Audio
from IPython.utils import io
from synthesizer.inference import Synthesizer
from encoder import inference as encoder
from vocoder import inference as vocoder
from pathlib import Path
import numpy as np
import librosa
encoder_weights = Path("encoder/saved_models/pretrained.pt")
vocoder_weights = Path("vocoder/saved_models/pretrained/pretrained.pt")
syn_dir = Path("synthesizer/saved_models/logs-pretrained/taco_pretrained")
encoder.load_model(encoder_weights)
synthesizer = Synthesizer(syn_dir)
vocoder.load_model(vocoder_weights)

In [ ]:
import soundfile as sf
text = "time to come to twenty twentyone with a better vocoder"
in_fpath = Path("../audio.wav")
original_wav, sampling_rate = librosa.load(in_fpath)
preprocessed_wav = encoder.preprocess_wav(original_wav, sampling_rate)
embed = encoder.embed_utterance(preprocessed_wav)

def synthesis(text, embed):
  with io.capture_output() as captured:
    specs = synthesizer.synthesize_spectrograms([text], [embed])
  generated_wav = vocoder.infer_waveform(specs[0])
  generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
  sf.write('generated.wav', generated_wav, synthesizer.sample_rate)
  

synthesis(text, embed)


In [ ]:
import IPython
IPython.display.Audio('generated.wav')

In [50]:
data, sr = librosa.load('tmp.wav', 44100)
tmp10sec = 44100*5
x = 1100000

In [52]:
info = data[x-tmp10sec:x+8000]
info = librosa.resample(info, 44100, 16000)
sf.write('trump.wav', info, 16000)

In [54]:
def voice_conversion(source_file, target_file):
  text = recognise_quartznet(source_file)
  original_wav, sampling_rate = librosa.load(target_file)
  preprocessed_wav = encoder.preprocess_wav(original_wav, sampling_rate)
  embed = encoder.embed_utterance(preprocessed_wav)
  result = synthesis(text, embed)
  return result

voice_conversion('beyonce.wav', 'trump.wav')


{| ████████████████ 66500/67200 | Batch Size: 7 | Gen Rate: 8.1kHz | }

In [48]:
data, sr = librosa.load('../audio.wav', 44100)
data = librosa.resample(data, 44100, 16000)
data = data[:16000*5]
sf.write('beyonce.wav', data, 16000)

In [ ]:
IPython.display.Audio('beyonce.wav')

In [ ]:
IPython.display.Audio('generated.wav')

In [79]:
cd /content/Real-Time-Voice-Cloning/

/content/Real-Time-Voice-Cloning


In [80]:
token=''
import telebot
from urllib.request import urlretrieve
import librosa
import soundfile as sf
i = 0


def convert_to_wav(filename):
  data, sr = librosa.load(filename)
  sf.write('tmp.wav', data, sr)
  return 'tmp.wav'

bot = telebot.TeleBot(token)
@bot.message_handler(content_types=["audio", "document", "voice"])
def handle(message, i=10): 
    try:
      file_info = bot.get_file(message.voice.file_id)
      downloaded_file = bot.download_file(file_info.file_path)
      with open('new_file.ogg', 'wb') as new_file:
        new_file.write(downloaded_file)
      new_name = convert_to_wav('new_file.ogg')
      print('saved at new_file.ogg')
      voice_conversion('tmp.wav', 'beyonce.wav')
      bot.send_audio(chat_id=message.chat.id, audio=open('generated.wav', 'rb'))

    except:
      bot.send_message(chat_id=message.chat.id, text='error')

In [ ]:
bot.polling()